In [231]:
import matplotlib.pyplot as plt

import os

import tensorflow as tf


In [ ]:
!wget https://motchallenge.net/data/MOT16.zip

--2021-11-24 16:20:03--  https://motchallenge.net/data/MOT16.zip
Resolving motchallenge.net (motchallenge.net)... 131.159.19.34, 2a09:80c0:18::1034
Connecting to motchallenge.net (motchallenge.net)|131.159.19.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1954509127 (1.8G) [application/zip]
Saving to: ‘MOT16.zip’

MOT16.zip            83%[===============>    ]   1.52G  11.8MB/s    eta 33s    

In [12]:
!mv MOT16.zip data


In [77]:
data = tf.data.Dataset.list_files("data/train/*/img*/*", shuffle=False)

In [78]:
def get_video_and_frame(file_path):
    path_elems = tf.strings.split(file_path, os.path.sep)
    video = int(tf.strings.substr(path_elems[2], 6, 2))
    frame = int(tf.strings.substr(path_elems[-1], 0, 6))
    return video, frame

In [ ]:
video_frame_object

In [ ]:
def get_objects(file_path):
    video_frame_objects = {0:(1,2,3,4)}
    gt = tf.data.Dataset.TextLineDateset("data/train/MOT16-02/gt/gt.txt") # I could select the right one with the path of the other one
    video, frame = get_video_and_frame(file_path)
    for x,w,w,h in video_frame_objects[0]:
        continue
    return tf.concat([])

In [79]:
data = data.map(get_video_and_frame)

In [80]:
for i in data.take(3):
    print(i)

(<tf.Tensor: shape=(), dtype=int32, numpy=2>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)
(<tf.Tensor: shape=(), dtype=int32, numpy=2>, <tf.Tensor: shape=(), dtype=int32, numpy=2>)
(<tf.Tensor: shape=(), dtype=int32, numpy=2>, <tf.Tensor: shape=(), dtype=int32, numpy=3>)


In [126]:
data = tf.data.Dataset.list_files("data/train/*/gt/*", shuffle=False)

In [131]:
def get_gt(file_path):
    video = int(tf.strings.substr(tf.strings.split(file_path, os.sep)[2], -2, 2))
    return tf.io.read_file(file_path), video
#     return tf.data.TextLineDataset(file_path)

In [133]:
for i in data.map(get_gt).batch(10).take(1): 
    print(i)
    print("\n")
    

In [387]:
feature_names = ["f"+str(i+1) for i in range(9)]
record_defaults = [-1.]*9

def preprocess_image(filename):
    # source: https://keras.io/examples/vision/siamese_network/
    target_shape = (200,200)
    image_string = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.convert_to_tensor(image)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, target_shape)
    return image


def decode_csv(line):
    parsed_line = tf.io.decode_csv(line, record_defaults)
    features = tf.stack(parsed_line)  # Stack features so that you can later vectorize forward prop., etc.
    batch_to_return = features
    return batch_to_return

def get_object(image, coords):
    x1 = tf.cast(coords[0], tf.int32)
    x2 = tf.cast(tf.add(coords[0],coords[2]), tf.int32)
    y1 = tf.cast(coords[1], tf.int32)
    y2 = tf.cast(tf.add(coords[1],coords[3]), tf.int32)
#     return image[x1:x2,y1:y2,:]
    return image
#     return x1,x2,y1,y2

def get_triplet(gt_line):
#     frame = gt_line[0:] 
#     id_ = gt_line[1]
#     x = gt_line[2]
#     y = gt_line[3]
#     w = gt_line[4]
#     h = gt_line[5]
    # get the video from the filename
    video = 2
#     filename = "data/train/MOT16-{:0>2}/img1/{:0>6}.jpg".format(str(video),str(frame))
#     print(tf.strings.format("data/train/MOT16-{}/img1/{}.jpg", (video, frame)))
    # TODO: need to figure out a more flexible way of formatting this string
    filename = tf.strings.format("data/train/MOT16-0{}/img1/00000{}.jpg",(video, tf.cast(gt_line[0], tf.int32)))
    image = preprocess_image(filename)
    image = get_object(image, gt_line[2:6])
    return image

In [388]:
# dataset = tf.data.Dataset.from_tensor_slices(["data/train/MOT16-02/gt/gt.txt","data/train/MOT16-04/gt/gt.txt"])

dataset = tf.data.Dataset.from_tensor_slices(["data/train/MOT16-02/gt/gt.txt"])
dataset = dataset.flat_map(lambda filename: tf.data.TextLineDataset(filename).map(decode_csv))
dataset = dataset.map(get_triplet)

# dataset = dataset.batch(2)

for i in dataset.take(4):
#     print(i[0], i[1], i[2], i[3])
    print(i)
#     plt.imshow(i)
#     plt.show()


tf.Tensor(
[[[0.21176472 0.21960786 0.16862746]
  [0.21568629 0.22352943 0.17254902]
  [0.21960786 0.227451   0.1764706 ]
  ...
  [0.39019564 0.37843093 0.41372505]
  [0.3803922  0.37647063 0.40784317]
  [0.37647063 0.37254903 0.4039216 ]]

 [[0.21647061 0.22431375 0.17333335]
  [0.21803923 0.22588237 0.17490196]
  [0.2219608  0.22980393 0.17882353]
  ...
  [0.3949015  0.3831368  0.41843092]
  [0.3803922  0.37647063 0.40784317]
  [0.37411767 0.37019607 0.40156865]]

 [[0.21960786 0.227451   0.1764706 ]
  [0.21960786 0.227451   0.1764706 ]
  [0.22352943 0.23137257 0.18039216]
  ...
  [0.39803877 0.38627407 0.42156819]
  [0.3803922  0.37647063 0.40784317]
  [0.37254903 0.36862746 0.40000004]]

 ...

 [[0.24705884 0.27450982 0.29803923]
  [0.24313727 0.27058825 0.29411766]
  [0.24901962 0.26862746 0.29215688]
  ...
  [0.20000002 0.227451   0.2509804 ]
  [0.20352904 0.22313689 0.23490159]
  [0.18313755 0.20274541 0.21451011]]

 [[0.2520784  0.27090174 0.2923918 ]
  [0.25207856 0.26933327 0

In [344]:
#source: https://stackoverflow.com/questions/49525056/tensorflow-python-reading-2-files/49548224#49548224

feature_names = ['f1','f2','f3','f4','f5']
record_defaults = [[""], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]


def decode_csv(line):
    parsed_line = tf.decode_csv(line, record_defaults)
    label =  parsed_line[-1]      # label is the last element of the list
    del parsed_line[-1]           # delete the last element from the list
    del parsed_line[0]            # even delete the first element bcz it is assumed NOT to be a feature
    features = tf.stack(parsed_line)  # Stack features so that you can later vectorize forward prop., etc.
    #label = tf.stack(label)          #NOT needed. Only if more than 1 column makes the label...
    batch_to_return = features, label
    return batch_to_return

filenames = tf.placeholder(tf.string, shape=[None])
dataset5 = tf.data.Dataset.from_tensor_slices(filenames)
dataset5 = dataset5.flat_map(lambda filename: tf.data.TextLineDataset(filename).skip(1).map(decode_csv))
dataset5 = dataset5.shuffle(buffer_size=1000)
dataset5 = dataset5.batch(7)
iterator5 = dataset5.make_initializable_iterator()
next_element5 = iterator5.get_next()

# Initialize `iterator` with training data. 
training_filenames = ["train_data1.csv", #TODO: I know w=which ones
                      "train_data2.csv"]

# Initialize `iterator` with validation data.
validation_filenames = ["dev_data1.csv"] #TODO: I know w=which ones

with tf.Session() as sess:
    # Train 2 epochs. Then validate train set. Then validate dev set.
    for _ in range(2):     
        sess.run(iterator5.initializer, feed_dict={filenames: training_filenames})
        while True:
            try:
                features, labels = sess.run(next_element5)
                # Train...
                print("(train) features: ")
                print(features)
                print("(train) labels: ")
                print(labels)  
            except tf.errors.OutOfRangeError:
                print("Out of range error triggered (looped through training set 1 time)")
                break

    # Validate (cost, accuracy) on train set
    print("\nDone with the first iterator\n")

    sess.run(iterator5.initializer, feed_dict={filenames: validation_filenames})
    while True:
        try:
            features, labels = sess.run(next_element5)
            # Validate (cost, accuracy) on dev set
            print("(dev) features: ")
            print(features)
            print("(dev) labels: ")
            print(labels)
        except tf.errors.OutOfRangeError:
            print("Out of range error triggered (looped through dev set 1 time only)")
            break 

AttributeError: module 'tensorflow' has no attribute 'placeholder'